# Agents o Assistants

Sabemos que los LLMs son grandes cajas negras que interpretan el lenguaje natural y nos pueden dar respuestas tambien en nuestro lenguaje de acuerdo a la informacion que tienen comprimida adentro de acuerdo a su entrenamiento.

Que pasa si a estos LLMs ademas les damos herramientas para que puedan obtener informacion actualizada y verificada del mundo real?

## OpenAI Functions and LangChain Tools

Veamos como podemos darles estas herramientas a los LLMs. En OpenAI les llaman functions y en LangChain Tools:

In [ ]:
!pip install openai --upgrade --quiet
!pip install langchain --upgrade --quiet

In [2]:
import openai

api_key="INGRESA AQUI TU TOKEN DE OPENAI"

In [3]:
#@title [Opcional] cargar el token desde colab para que no se vea al dar clases {display-mode:"form"}
if api_key == "INGRESA AQUI TU TOKEN DE OPENAI":
    from google.colab import userdata
    api_key = userdata.get('OPENAI_TOKEN')
    openai.api_key = api_key

Mounted at /content/drive


In [9]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMMathChain
from langchain.chains import LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
import re

In [25]:
# Define which tools the agent can use to answer user queries
tools = [
    Tool(
        name="math",
        func=LLMMathChain.from_llm(OpenAI(openai_api_key=api_key)),
        description="util cuando quieres realizar operaciones matematicas"
    )
]

In [37]:
# Set up the base template
template = """Contesta la siguiente pregunta lo mejor que puedas.
Tienes acceso a las siguientes herramientas:
{tools}
Usa el siguiente formato:
Pregunta: la pregunta original de entrada que debes responder
Analisis: siempre debes analizar que hacer
Accion: la accion a realizar, debe ser una de [{tool_names}]
Entrada Accion: la entrada de la accion
Observacion: el resultado de la accion
Analisis: Ya se la respuesta final
Respuesta Final: la respuesta final a la pregunta original de entrada
Pregunta: {input}
{agent_scratchpad}
"""

In [38]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\Observacion: {observation}\Analisis: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [39]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

In [40]:
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Respuesta Final:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Respuesta Final:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Accion\s*\d*\s*:(.*?)\nEntrada\s*\d*\s*Accion\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise OutputParserException(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [41]:
output_parser = CustomOutputParser()

In [42]:
llm = OpenAI(openai_api_key=api_key, temperature=0)

In [43]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [44]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservacion:"],
    allowed_tools=tool_names
)

In [45]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.run("Cuantos es tres mas IV?")